In [65]:
import re
import pandas as pd

def read_whatsapp_chat(file_path:str):
    encryption_message = "Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more."
    media_pattern = "<Media omitted>"
    email_pattern = r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}'
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    edited_message = "<This message was edited>"
    deleted_message = "You deleted this message"
    null_message = "null"
    created_group_message = "created group"
    added_you_to_group_message = "added you"
    tagging_pattern = r'@[\w]+'

    with open(file_path, 'r', encoding = 'utf-8' ) as f:
        lines = f.readlines()

    # Apply filters to remove unwanted lines
    filtered_lines = []
    for line in lines:
        if (
            encryption_message not in line and
            deleted_message not in line and
            null_message != line.split(" ")[-1] and
            media_pattern not in line and
            created_group_message not in line and
            added_you_to_group_message not in line and
            not re.search(email_pattern, line) and
            not re.search(url_pattern, line)
        ):
            line = line.replace(edited_message, "").strip()
            line = re.sub(tagging_pattern, "", line).strip()
            filtered_lines.append(line)

    # Normalize content:
    content = '\n'.join(filtered_lines)
    # Replace narrow no-break space (iOS specific)
    content = content.replace('\u202f', ' ')
    # Remove square brackets if they surround the timestamp (only for iOS)
    content = re.sub(
        r'\[(\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2}(?::\d{2})?\s?[APap][Mm])\]',
        r'\1',
        content
    )
    # Remove LRM and RLM characters (Left-to-Right Mark and Right-to-Left Mark)
    content = content.replace('\u200E', '').replace('\u200F', '')

    # Updated regex pattern to match both iOS and Android WhatsApp exports.
    pattern = r'(\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2}(?::\d{2})?(?:\s?[APap][Mm])?)\s?(?:-|\~)?\s?(.*?): (.*?)(?=\n\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2}|$)'
    messages = re.findall(pattern, content, re.DOTALL)

    df = pd.DataFrame(messages,columns=['timestamp','sender','message'])
    timestamps = []
    for timestamp in df['timestamp']:
        try:
            timestamp = pd.to_datetime(
                timestamp, format='mixed', errors='coerce')
        except Exception as e:
            print(f"Error parsing timestamp '{timestamp}': {e}")
            timestamp = pd.NaT
        timestamps.append(timestamp)

    df['timestamp'] = timestamps
    return df

In [66]:
from pathlib import Path

all_chats = {}
data_directory = Path("./data/")

for file in data_directory.glob('*.txt'):
    print(file)
    file_name = file.stem
    all_chats[file_name] = read_whatsapp_chat(file)
    print(all_chats[file_name])

data/WhatsApp Chat with Ankit Babua.txt
               timestamp                                             sender  \
0    2021-02-25 20:20:00  Messages and calls are end-to-end encrypted. O...   
1    2021-02-25 20:20:00                                        Ankit Babua   
2    2021-03-14 21:56:00                                        Ankit Babua   
3    2021-03-14 21:59:00                                       sumit mandal   
4    2021-03-14 22:00:00                                       sumit mandal   
...                  ...                                                ...   
1683 2025-05-31 23:29:00                                       sumit mandal   
1684 2025-05-31 23:30:00                                        Ankit Babua   
1685 2025-05-31 23:32:00                                       sumit mandal   
1686 2025-06-01 10:03:00                                        Ankit Babua   
1687 2025-06-01 10:03:00                                        Ankit Babua   

           

In [67]:
text_sequence = ""
for file_name in all_chats.keys():
    text_sequence += " ".join(all_chats[file_name]['message'].values)

len(text_sequence)

294177

In [68]:
with open("./output/combined_text.txt", "w", encoding="utf-8") as f:
    f.write(text_sequence)

In [69]:
!git init

Reinitialized existing Git repository in /Users/smandal14/work_aig/aig/Kendra-Bot/whatsapp_own_bot_training/.git/


In [60]:
!git add .

In [61]:
!git commit -m "Data preprocessing"

[main (root-commit) 4d75e6d] Data preprocessing
 Committer: Mandal <smandal14@TAGLDP23483.local>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly:

    git config --global user.name "Your Name"
    git config --global user.email you@example.com

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 2 files changed, 514 insertions(+)
 create mode 100644 .gitignore
 create mode 100644 whatsapp_cleaning_data_preprocessing.ipynb


In [62]:
!git branch -M main

In [63]:
!git remote add origin git@github.com-personal:sumit-mandal/llm-training-from-scratch.git

In [64]:
!git push -u origin main


Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 4.90 KiB | 2.45 MiB/s, done.
Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
To github.com-personal:sumit-mandal/llm-training-from-scratch.git
 * [new branch]      main -> main
branch 'main' set up to track 'origin/main'.
